In [6]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F

import os
from pathlib import Path
import numpy as np
from glob import glob
import glob
import random

from torchvision import transforms
from collections import defaultdict

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [7]:
class GeoImageDataset(Dataset):
    def __init__(self, img_dir: Path, mask_dir:Path, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.img_files = os.listdir(self.img_dir)
        self.mask_files = os.listdir(self.mask_dir)
        self.transform = transform
        

    def __len__(self):
        return len(self.img_files)
        
    def __getitem__(self, idx):
        # Load image
        img_path = self.img_dir / self.img_files[idx]
        # mask and img_file have so far the same name
        mask_path = self.mask_dir / self.img_files[idx]
        img = torch.load(img_path)
        # converts bool mask into integer (0/1)
        mask = torch.load(mask_path).long()
        # Apply transform (if any)
        if self.transform:
            img = self.transform(img)
        
        return img, mask #, img_path, mask_path

In [8]:
root = Path(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\data_splitted')
train_img_dir = root / "train/images"
train_mask_dir = root / "train/masks"

val_img_dir = root / "val/images"
val_mask_dir = root / "val/masks"

test_img_dir = root / "test/images"
test_mask_dir = root / "test/masks"

train_dataset = GeoImageDataset(train_img_dir, train_mask_dir)
val_dataset = GeoImageDataset(val_img_dir, val_mask_dir)
test_dataset = GeoImageDataset(test_img_dir, test_mask_dir)

In [9]:
batch_size = 32
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

test_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True
)

In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [25]:
from unet import Unet
model = Unet(encoder_name='efficientnet-b0', encoder_weights='imagenet', in_channels=4, activation='sigmoid', classes=1).to(device)

(4, 32, 24, 40, 112, 320)


In [15]:
from unet import Unet
model = Unet(encoder_name='efficientnet-b0', encoder_weights='imagenet', in_channels=4, activation='sigmoid', classes=1).to(device)
x = train_dataset[0]
with torch.no_grad():
    traced_cell = torch.jit.trace(model, (x))
torch.jit.save(traced_cell, "model.pth")

TypeError: SegmentationModel.forward() takes 2 positional arguments but 3 were given

In [17]:
from unet import Unet
model = Unet(encoder_name='efficientnet-b0',encoder_weights='imagenet', in_channels=4, activation='sigmoid', classes=1).to(device)
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('model_scripted.pt') # Save

RuntimeError: Can't redefine method: forward on class: __torch__.unet.decoders.model.___torch_mangle_1.Unet (of Python compilation unit at: 0000027DDBD58CB0)